In [1]:
# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import math
import torch

# wrapper for wikipedia API
import wikipedia

# scraping of web articles
from newspaper import Article
# google news scraping
from GoogleNews import GoogleNews

# graph visualization
from pyvis.network import Network

# show HTML in notebook
import IPython

from keybert import KeyBERT

from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
import re
from typing import List, Tuple, Union

import spacy

import inspect
import os

import ollama
from tqdm import tqdm
import time
import re
import sys
import pandas as pd

module_path = '../core'
sys.path.append(module_path)

from SemanticSimilarity import SemanticSimilarity
from KeywordExtractor import KeywordExtractor
from TripletGenerator import TripletGenerator

/home/niamatzawad/miniconda3/envs/phd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)     # Display all rows


In [3]:
class KG:
    def __init__(self,):
        self.triplets = []

    def add_triplets(self, triplets : List):
        self.triplets.extend(triplets)


In [4]:
def retrieve_text_from_filepath(filepath):
    with open(filepath,'r') as f:
        t = f.read()
        return t


In [5]:
%%time
# folder_path = "/home/niamatzawad/niamatzawad/Datasets/UTDBox/demoNY/unsplitted"
folder_path = "/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4"

s_demo = SemanticSimilarity()
kwe_demo = KeywordExtractor()
kg_demo = KG()

folder_objects = os.listdir(folder_path)
for i, filename in enumerate(folder_objects):
    if os.path.isfile(os.path.join(folder_path, filename)):
        with open(os.path.join(folder_path, filename), 'r') as file:
            text = file.read()
            text_splitted = [text[i * 1900:(i+1) *1900] for i in range(len(text)//1900)]
            start_text_time = time.time()
            for text_s in tqdm(text_splitted):
                start_subtext_time = time.time()
                try:
                    named_entities = kwe_demo.extract_named_entities(text_s)
                except Exception as e:
                    print(e)
                    continue
                s_demo.generate_reference(named_entities)
                t = TripletGenerator(text_s, s_demo, kwe_demo, os.path.join(folder_path, filename), named_entities=named_entities)
                triplets = t.get_triplets()
                kg_demo.add_triplets(triplets)
                print(f"Time to create triplets for current subtext - {time.time() - start_subtext_time}")
                
        print(f"Finished processing {i+1}/ {len(folder_objects)} files")
        print(f"Total time taken for current text - {time.time() - start_text_time}")
        print("---")
        


 50%|██████████████████████▌                      | 1/2 [00:12<00:12, 12.91s/it]

{'model': 'mistral', 'created_at': '2024-05-25T00:09:35.257821007Z', 'response': ' Triplet 1: (The Ritz-Carlton Hotel Company, operates, The Ritz-Carlton Jakarta)\nText: The Ritz-Carlton Jakarta is operated by The Ritz-Carlton Hotel Company.\n\nTriplets 2 and 3: (The Oberoi Group, is famous for, involvement in hotels)\nText: The Oberoi family is famous for its involvement in hotels, namely through The Oberoi Group.\n\nTriplet 4: (Shivaay, portrays, a heir of the Oberoi family)\nText: Nakuul Mehta respectively portrays Shivaay, one of the heirs of the Oberoi family.\n\nTriplet 5: (The Eppley Hotel Company, built, The Hotel Tallcorn)\nText: The Hotel Tallcorn was built by the Eppley Hotel Company.\n\nTriplet 6: (local citizens, contributed, $120,000)\nText: local citizens contributed $120,000 to ensure the successful completion of this seven-story hotel.\n\nNone: Ishqbaaaz, a drama television series\nJW Marriott Hotel, a hotel adjacent to The Ritz-Carlton Jakarta\nMega Kuningan, a neighb

100%|█████████████████████████████████████████████| 2/2 [00:22<00:00, 11.00s/it]


{'model': 'mistral', 'created_at': '2024-05-25T00:09:44.464423966Z', 'response': " Entity: Hotels & Resorts (India's second-largest hotel company)\nRelation type: owns/manages\nEntities: India, Sri Lanka, Nepal, Egypt, Australia, Hungary\nText: Hotels & Resorts, India's second-largest hotel company, with 35 hotels in India, Sri Lanka, Nepal, Egypt, Australia and Hungary.\n\nEntity: Hotel Bond\nRelation type: built by\nEntity: Harry S. Bond (hotelier)\nText: It is located near Bushnell Park, and was considered the grandest hotel in Hartford during its heyday. The second section is a 12 story building attached to the 6 story first section. A Statler Hotel opened in the area in 1954, creating competition, and the Bond Hotel company declared bankruptcy shortly after that. It was bought by the California-based Masaglia Hotel chain, which began an incremental renovation program. In 1964 it was sold to a Cincinnati, Ohio investment group which announced extensive renovation plans. However, th

 25%|███████████▎                                 | 1/4 [00:05<00:16,  5.35s/it]

...

{'model': 'mistral', 'created_at': '2024-05-25T00:10:23.633304647Z', 'response': ' Entity: Henry Miller\nText: Moloch: or, This Gentile World is a semi-autobiographical novel written by Henry Miller... It went unpublished until 1992, 65 years after it was written and 12 years after Miller’s death. (Relation type: wrote)\n\nEntity: James Henry Deakin senior\nText: The Launceston by-election of 1874 was fought due to the void Election of the incumbent Conservative MP, James Henry Deakin senior. (Relation type: served as)\n\nEntity: James Henry Deakin junior\nText: It was won by the Conservative candidate James Henry Deakin junior. (Relation type: won)\n\nEntity: Anaïs Nin\nText: Incest: From a Journal of Love: The Unexpurgated Diary of Anaïs Nin (1932–1934) is a 1992 non-fiction book by Anaïs Nin... It features Nin\'s relationships with writer Henry Miller, his wife June Miller, the psychoanalyst Otto Rank, her father Joaquín Nin, and her husband Hugh Parker Guiler. She also copied some 

 50%|██████████████████████▌                      | 1/2 [00:03<00:03,  3.26s/it]

{'model': 'mistral', 'created_at': '2024-05-25T00:10:26.674116841Z', 'response': ' (Radio City, was started in, Mumbai)\n(Radio City, was started in, Bengaluru)\n(Radio City, was started in, Lucknow)\n(Radio City, was started in, New Delhi)\n(Albanian Football Federation, was founded on, June 6, 1930)\n(Albania, joined FIFA, during the 12–16 June convention )\n(Echosmith, signed to Warner Bros. Records, in May 2012)', 'done': True, 'done_reason': 'stop', 'context': [733, 16289, 28793, 28705, 10290, 28747, 28777, 5067, 272, 19810, 5936, 19036, 2990, 647, 464, 507, 515, 647, 464, 28719, 2915, 1585, 647, 464, 28726, 980, 282, 19099, 647, 464, 28714, 1384, 3306, 647, 464, 1095, 882, 5365, 647, 464, 279, 647, 464, 5942, 826, 20882, 647, 464, 1839, 1520, 28710, 647, 464, 3225, 17212, 377, 270, 6701, 647, 464, 19036, 2990, 647, 464, 11009, 299, 4306, 28710, 9508, 647, 464, 1237, 6480, 5086, 647, 464, 375, 11162, 306, 23856, 647, 464, 17998, 647, 464, 282, 3627, 515, 647, 464, 1237, 389, 3627,

100%|█████████████████████████████████████████████| 2/2 [00:10<00:00,  5.25s/it]

Time to create triplets for current subtext - 7.236973524093628
Finished processing 4/ 4 files
Total time taken for current text - 10.503974914550781
---
CPU times: user 29.2 s, sys: 5.38 s, total: 34.6 s
Wall time: 1min 13s


In [6]:
def get_subjects_from_triplets(triplets):
    return list(set([i[0] for i in triplets]))
get_subjects_from_triplets(kg_demo.triplets)

['e wechsler',
 'milhouse',
 'newspaper',
 'lisa marie simpson',
 'shivaay',
 'roening',
 'salem college south',
 'the ritz - carlton jakarta',
 'the eppley hotel company',
 'echosmith',
 'local citizens',
 'albania',
 'hk kremenchukfirst for women',
 'Vowell',
 'nthe oberoi family',
 'faber',
 'ukrainian',
 'goertz',
 'matt groening',
 'arthur county courthouse',
 'the song',
 'r jay simpson',
 't s . arthur',
 'james l brooks',
 'radio city',
 'the simpsons',
 'john ortved',
 'dan sallit']

In [7]:
kg_demo.triplets

[('the ritz - carlton jakarta',
  ' operates',
  'the ritz - carlton jakarta',
  'The Ritz-Carlton Jakarta is operated by The Ritz-Carlton Hotel Company.'),
 ('the ritz - carlton jakarta',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('nthe oberoi family',
  ' is famous for',
  'hotel',
  'The Oberoi family is famous for its involvement in hotels, namely through The Oberoi Group.'),
 ('nthe oberoi family',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('shivaay',
  ' portrays',
  'nthe oberoi family',
  'Nakuul Mehta respectively portrays Shivaay, one of the heirs of the Oberoi family.'),
 ('shivaay',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('the eppley hotel company',
  ' built',
  'hotel',
  'The Hotel Tallcorn was built by the Eppley Hotel Company.'),
 ('the eppley hotel company',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_

In [8]:
# np.save('disambiguated_reference_list_hotpot_100.npy', s.disambiguated_reference_list)    # .npy extension is added if not given
# disabiguated_reference_liit = np.load('disambiguated_reference_list.npy')

In [9]:
# np.save('embedded_reference_list_hotpot_100.npy', s.embedded_reference)    # .npy extension is added if not given
# embedded_reference = np.load('embedded_reference.npy')

In [10]:
s_demo.disambiguated_reference_list

['the ritz - carlton jakarta',
 'hotel',
 'skyscraper',
 'mega kuningan',
 'the sister',
 'the complex',
 'the airlangga apartment',
 'nthe oberoi family',
 'star plus',
 'nakuul mehta',
 'leenesh mattoo',
 'shivaay',
 'omkara',
 'rudra',
 'three',
 'annika',
 'surbhi chandna',
 'marshalltown , iowa',
 'local citizens',
 'this',
 'the eppley hotel company',
 'the sheraton corporation',
 'the tallcorn towers apartments',
 'the national register of historic places',
 'rudraey oberoi']

In [8]:
from py2neo import Graph, Node, Relationship



# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

In [9]:
from py2neo import Graph, Node, Relationship



# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

triplets = kg_demo.triplets

# Create the nodes and relationships
for t in triplets:
    # Create the subject node
    subject, predicate, obj = t[0],t[1],t[2]
    context = None
    if len(t) == 4:
        context = t[3]
    
    try:
        n = graph.run(f"MATCH (n:`{subject}`) RETURN n;").data()
    except Exception as e:
        print(f"Problem with subject {subject}")
        continue

    if n:
        subject_node = n[0]["n"]
    else:
        subject_node = Node(subject, id=subject)
        graph.create(subject_node)

    if predicate == "filepath":
        # obj = re.sub("NYT Files", "'NYT Files'",obj)
        obj = "```"+obj+"```"
        context = retrieve_text_from_filepath(obj[3:-3])
        n = graph.run(f"MATCH (n:{obj}) RETURN n;").data()
        if n:
            # print(n[0]["n"])
            object_node = n[0]["n"]
        else:
            # obj = re.sub("'NYT Files'", "NYT Files",obj)[1:-1]
            
            obj = obj[2:-2]
            object_node = Node(obj, id=obj)
            graph.create(object_node)
    else:
        try:
            n = graph.run(f"MATCH (n:`{obj}`) RETURN n;").data() #I had the assumption that subjects are being linked to wrong entities. Seems that is not the case as it exactly matches object if it exists and not the most similar object
        except Exception as e:
            print(f"Problem with object - {obj}")
            continue
            
        if n:
            object_node = n[0]["n"]
        else:
            object_node = Node(obj, id=obj, text=context)
            graph.create(object_node)

    relationship = Relationship(subject_node, predicate, object_node, text=context)
    graph.merge(relationship)

print("Triplets added to the Neo4j database.")

Triplets added to the Neo4j database.


In [13]:
s = SemanticSimilarity()
s.disambiguated_reference_list

[]

In [14]:
np.save('disambiguated_reference_list_kg_demo_hotpot_100.npy', s_demo.disambiguated_reference_list)    # .npy extension is added if not given
np.save('embedded_reference_kg_demo_hotpot_100.npy', s_demo.embedded_reference)    # .npy extension is added if not given


In [15]:
for i in kg_demo.triplets:
    if i[0] == "dua_lipa":
        print(i)
        break

In [16]:
kg_demo.triplets

[]

In [11]:
triples = []
for k in kg_demo.triplets:
    t = list(k)
    if len(t) == 3:
        t = t + [""]
    if len(t) > 4:
        continue
    triples.append(t)

